In [117]:
Train_Model = True

In [118]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [119]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [120]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [121]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [122]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [123]:
Video_Dataset_Test = "Video_Dataset_Test"
Video_file = "00335.mp4"

Video = os.path.join( Video_Dataset_Test , Video_file )

In [124]:
cap = cv2.VideoCapture(Video)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
    # while True:

        # Read feed
        ret, frame = cap.read()
        
        # If the frame was not read successfully, it means we've reached the end of the video
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # if not Train_Model:
        #     break
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [125]:
# draw_landmarks(frame, results)

In [126]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

## 3. Extract Keypoint Values

In [127]:
# len(results.left_hand_landmarks.landmark)
# len(results)

In [128]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [129]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [130]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [131]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [132]:
result_test = extract_keypoints(results)
result_test

array([ 0.49358568,  0.24614742, -0.83555347, ...,  0.        ,
        0.        ,  0.        ])

In [133]:
np.save('0', result_test)
np.load('0.npy')

array([ 0.49358568,  0.24614742, -0.83555347, ...,  0.        ,
        0.        ,  0.        ])

## 4. Setup Folders for Collection

In [134]:
# doc = "Docs"
# text_file = "actions.txt"
# with open(os.path.join(doc, text_file), 'r') as f:
#     action_str = f.read() 

# # list to np array
# action_names = np.array(action_str.split('.'))

action_names = np.array( ["Test_Video"] )

actions = np.arange(len(action_names))
print(action_names)
print(actions)

['Test_Video']
[0]


In [135]:
# Path for exported data, numpy arrays
DATA_PATH_TEST = os.path.join('MP_Data_Test') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 1

# Videos are going to be 30 frames in length
sequence_length = 1

# Folder start
start_folder = 1

In [136]:
# Remove the directory if it exists
import shutil
if os.path.exists(DATA_PATH_TEST) and Train_Model:
    shutil.rmtree(DATA_PATH_TEST)

In [137]:
# Create the directory
if Train_Model:
    os.makedirs(DATA_PATH_TEST , exist_ok=True)

    for action in actions:
        os.makedirs(os.path.join(DATA_PATH_TEST , str(action)) , exist_ok=True)


    for action in actions: 
        # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH_TEST, action))).astype(int))
        for sequence in range(1,no_sequences+1):
            try: 
                os.makedirs(os.path.join(DATA_PATH_TEST, str(action), str(sequence)))
            except:
                pass

## 5. Collect Keypoint Values for Training and Testing

In [138]:
cap = cv2.VideoCapture(Video)

Break_capture = False
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1, no_sequences+1):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_names[action], sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_names[action], sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH_TEST, str(action), str(sequence), str(frame_num))
                
                if Train_Model:
                    np.save(npy_path, keypoints)
                else:
                    Break_capture = True
                    break
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    Break_capture = True
                    break
                
            if Break_capture:
                break
        if Break_capture:
            break
        
                    
    cap.release()
    cv2.destroyAllWindows()

In [139]:
cap.release()
cv2.destroyAllWindows()

In [140]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [141]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [142]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{0: 0}

In [143]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH_TEST, str(action)))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH_TEST, str(action), str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [144]:
np.array(sequences).shape

(1, 1, 1662)

In [145]:
np.array(labels).shape

(1,)

In [146]:
X = np.array(sequences)
X.shape

(1, 1, 1662)

In [147]:
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

ValueError: With n_samples=1, test_size=0.05 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.